# 6. Example Use Case of Therlid Dataset

An example notebook to show how to use the Therlid dataset.  It starts with basic exploratory data analysis (EDA) to visualize the data and then builds a simple predictive model. It’s a practical example for anyone learning to work with the Therlid dataset

## Defines and Imports

In [1]:
# replace with own projectid to pull from bigquery
%env GOOGLE_CLOUD_PROJECT = aiwonglab

env: GOOGLE_CLOUD_PROJECT=aiwonglab


In [2]:
# install necessary packages
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
from google.cloud import bigquery
import matplotlib.pyplot as plt
import missingno as msno
import statsmodels.api as sm
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.cov_struct import Exchangeable
import patsy
import re
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


In [ ]:
# load data temperature data stored in google collab or your created version using pd.read_csv()
%%bigquery final_df --project spring-carving-271217

SELECT *
FROM `spring-carving-271217.test.merged_temp_pairs`

## EDA

In [ ]:
temporal = final_df['temp_c_temporal']
other = final_df['temp_c_reference']
source_db = final_df['source_db']

fig, ax = plt.subplots(figsize=(10, 6))

colors = {'mimic_iv':'red', 'eicu':'blue', 'eicu2':'green'}  # Replace with actual source_db categories
for source in final_df['source_db'].unique():
    subset = final_df[final_df['source_db'] == source]
    ax.scatter(subset['temp_c_temporal'],
               subset['temp_c_reference'],
               alpha=0.3, label=source, color=colors.get(source, 'gray'))

ax.axvline(x=36, color='black', linestyle='--')
ax.axvline(x=38, color='black', linestyle='--')
ax.axhline(y=36, color='black', linestyle='--')
ax.axhline(y=38, color='black', linestyle='--')
ax.set_title("Scatterplot of Temporal vs Reference Temperature by Source Database", fontsize=20)
ax.set_xlabel("Temporal Temperature (°C)", fontsize=20)
ax.set_ylabel("Reference Temperature (°C)", fontsize=20)


ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)
ax.legend(title='Source Database', title_fontsize=14, fontsize=14)

plt.show()

## Linear Model with Generalized Estimating Equations

In [ ]:
# using only the first pair for each subject
first_pair = final_df[final_df['pair_seq'].str.endswith('-1')]

# selected vars for model
vars = ['unique_hospital_admission_id', 'race_ethnicity', 'admission_age', 'sex_female', 'source_db', 'comorbidity_score_value', 'hidden_SIRS', 'sofa_day1']

selected_df = first_pair[vars]

selected_df

In [ ]:
# drop pairs that don't have sofa info
completed_pairs = selected_df[selected_df['sofa_day1'].notna()]
# convert hidden SIRS to float
completed_pairs['hidden_SIRS'] = completed_pairs['hidden_SIRS'].astype('float64')
completed_pairs

In [ ]:
# convert types to float64 to pass into statsmodel
completed_pairs = completed_pairs.astype({
    'admission_age': 'float64',
    'sex_female': 'float64',
    'comorbidity_score_value': 'float64'
})

In [ ]:
completed_pairs['race_ethnicity'] = completed_pairs['race_ethnicity'].astype('category')
# reorder to put 'white' as the reference category
completed_pairs['race_ethnicity'] = completed_pairs['race_ethnicity'].cat.reorder_categories(
    ['White', 'American Indian / Alaska Native', 'Asian', 'Black', 'Hispanic OR Latino', 'Native Hawaiian / Pacific Islander', 'Unknown'],
    ordered=False
)
# create matrices to plug inot the statsmodel
y, X = patsy.dmatrices('hidden_SIRS ~ race_ethnicity + C(sex_female) + source_db + admission_age + comorbidity_score_value + sofa_day1', data=completed_pairs, return_type='dataframe')


In [ ]:
groups = completed_pairs['unique_hospital_admission_id']

# define the GEE model with independence strucutre and grouping by hosptial admission id
model_gee_sirs = sm.GEE(y, X, groups=groups, cov_struct=sm.cov_struct.Independence(), family=sm.families.Binomial())
# fit the model
results = model_gee_sirs.fit()
# print the summary
print(results.summary())